In [1]:
from cc_parser.buckets import ALL_BUCKETS
from cc_parser.constants import CCType, CCHeader
from cc_parser.utils import *
from config import ExpenseManagerConstants
try:
    from cc_parser.my_buckets import ALL_BUCKETS as PRIVATE_BUCKETS
    ALL_BUCKETS.update(PRIVATE_BUCKETS)
except Exception as e:
    pass

In [3]:
# Sample to Load Individual Statement
# discover_tx_list = parseStatements(statements[0], CCType.DISCOVER)
# discover_bucket = groupTxInBuckets(discover_tx_list, ALL_BUCKETS, CCType.DISCOVER)

# Load All Statements
statements = loadStatements(ExpenseManagerConstants.EXPENSE_STATEMENT_PATHS)

for statement in statements.values():
    statement["statements"] = parseStatements(statement["statements"], statement["type"])
    statement["bucket"] = groupTxInBuckets(statement["statements"], ALL_BUCKETS, statement["type"])

merged_buckets, merged_tx = mergeCardBuckets( [ statement["bucket"] for statement in statements.values() ], filter_buckets=[], build_tuples=True )
merged_tx_copy = merged_tx
tx_list = formatTxTuples( sort_tx_by_date( sliceTxByDate(merged_tx_copy, '01/01/2021')) )
saveToCSV(tx_list)


In [ ]:
"""
################
READ & UPDATE tx_*.csv
update the last 2 columns
**, 1,
Update the above row to either {1,2,3}
(1, 'PERSONAL'), (2, 'SPLIT'), (3, 'NOT_SPLIT')
and the last field to any personal comments 

end result
**, 3, Local Trip
################
"""

In [ ]:
"""
1. Save to DB
2. 
"""

from cc_parser.db import DBLite
from splitwise.adapter import SplitwiseSDK
sql_driver = DBLite()
split_sdk = SplitwiseSDK()
split_sdk.setGroupId('{INSERT GROUP ID HERE}')

# Run Migrations
sql_driver.migrationsUp()

# Bulksert Transactions into DB
parsed_file_name = 'tx.csv'
tx_list = readCSV( parsed_file_name )
# sql_driver.insertTxList(tx_list)


# Filter Transactions categorized to be SPLIT {3}
non_split_transactions = sql_driver.getNonSplitPayments()

for [_ID, _HASH, _DATE, _TITLE, _CATEGORY, _AMOUNT,\
      _COMMENT, _CTYPE, _SPLIT_TYPE, _SPLITWISE_CATEGORY] in non_split_transactions:
    print(_TITLE)
#     resp = split_sdk.createExpense(_TITLE, str(_AMOUNT), _DATE, _SPLITWISE_CATEGORY)
#     print(_HASH, " -- ", resp["expenses"][0]["id"])
#     sql_driver.updateTx(_ID, _HASH, resp["expenses"][0]["id"])

sql_driver.close()